In [2]:
import pandas as pd
import mysql.connector

conn = mysql.connector.connect(username= "root",
                               host = "127.0.0.1",
                               password = "Admin@123",
                               database = "dataspark1")

cursor = conn.cursor()

In [3]:
df_customer = pd.read_csv('Customers.csv', encoding='latin1')

df_customer.columns = df_customer.columns.str.lower()
df_customer.columns = df_customer.columns.str.replace(" ","_")

from datetime import datetime

# Convert 'date_of_birth' to datetime
df_customer['birthday'] = pd.to_datetime(df_customer['birthday'])

# Define a function to calculate age
def calculate_age(birthdate):
    today = datetime.today()
    age = today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))
    return age

# Apply the function to calculate age for each customer
df_customer['age'] = df_customer['birthday'].apply(calculate_age)

df_customer

def age_segment(age):
    if age < 25:
        return 'young adult'
    elif age < 55:
        return 'middle age'
    else:
        return 'senior citizen'

# Apply the function to the 'age' column
df_customer['segment'] = df_customer['age'].apply(age_segment)


In [4]:
df_customer.head()

,customerkey,gender,name,city,state_code,state,zip_code,country,continent,birthday,age,segment
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,1939-07-03,85,senior citizen
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,1979-09-27,44,middle age
2,554,Female,Claire Ferres,WINJALLOK,VIC,Victoria,3380,Australia,Australia,1947-05-26,77,senior citizen
3,786,Male,Jai Poltpalingada,MIDDLE RIVER,SA,South Australia,5223,Australia,Australia,1957-09-17,66,senior citizen
4,1042,Male,Aidan Pankhurst,TAWONGA SOUTH,VIC,Victoria,3698,Australia,Australia,1965-11-19,58,senior citizen


In [7]:
df_customer.columns

Index(['customerkey', 'gender', 'name', 'city', 'state_code', 'state',
       'zip_code', 'country', 'continent', 'birthday', 'age', 'segment'],
      dtype='object')

In [22]:
df_customer[df_customer['state_code'] == "NA"]

,customerkey,gender,name,city,state_code,state,zip_code,country,continent,birthday,age,segment


In [ ]:
null_rows = df_customer[df_customer.isnull().any(axis=1)]
null_rows

In [32]:
duplicates = df_customer[df_customer['customerkey'].duplicated()]
duplicates

,customerkey,gender,name,city,state_code,state,zip_code,country,continent,birthday,age,segment


In [28]:
df_customer['state_code'] = df_customer['state_code'].fillna("NA")

In [34]:
df_customer.drop_duplicates(subset=['customerkey'], keep='first', inplace=True)

In [35]:
df_customer.shape

(15266, 12)

In [36]:
df_customer.isna().sum()

customerkey    0
gender         0
name           0
city           0
state_code     0
state          0
zip_code       0
country        0
continent      0
birthday       0
age            0
segment        0
dtype: int64

In [37]:
insert_customer_values = '''
    INSERT INTO customer (
        customerkey,
        gender,
        name,
        city,
        state_code,
        state,
        zip_code,
        country,
        continent,
        birthday,
        age,
        segment
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
'''

# Insert data into the customer table
for index, row in df_customer.iterrows():
    cursor.execute(insert_customer_values, (
        row['customerkey'],
        row['gender'],
        row['name'],
        row['city'],
        row['state_code'],
        row['state'],
        row['zip_code'],
        row['country'],
        row['continent'],
        row['birthday'],
        row['age'],
        row['segment']
    ))

conn.commit()

IntegrityError: 1062 (23000): Duplicate entry '301' for key 'customer.PRIMARY'

In [38]:
insert_customer_values = '''
    INSERT INTO customer (
        customerkey,
        gender,
        name,
        city,
        state_code,
        state,
        zip_code,
        country,
        continent,
        birthday,
        age,
        segment
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
        gender = VALUES(gender),
        name = VALUES(name),
        city = VALUES(city),
        state_code = VALUES(state_code),
        state = VALUES(state),
        zip_code = VALUES(zip_code),
        country = VALUES(country),
        continent = VALUES(continent),
        birthday = VALUES(birthday),
        age = VALUES(age),
        segment = VALUES(segment)
'''

# Insert data into the customer table
for index, row in df_customer.iterrows():
    cursor.execute(insert_customer_values, (
        row['customerkey'],
        row['gender'],
        row['name'],
        row['city'],
        row['state_code'],
        row['state'],
        row['zip_code'],
        row['country'],
        row['continent'],
        row['birthday'],
        row['age'],
        row['segment']
    ))

conn.commit()


DataError: 1406 (22001): Data too long for column 'state_code' at row 1